## Load the data

In [30]:
import os

embeddings_dir = "../data/embeddings/"
all_files = []
for file in os.listdir(embeddings_dir):
    if file.endswith(".json"):
        all_files.append(os.path.join(embeddings_dir, file))

print(f"Found {len(all_files)} embedding files.")

Found 5 embedding files.


In [31]:
import json

chunks = []
questions = []
chunk_id_counter = 0

for file_path in all_files:
    with open(file_path, "r") as f:
        data = json.load(f)

    file_chunks = data["chunks"]
    file_questions = data["question_answer_pairs"]

    old_to_new_id_map = {}

    # Update chunk IDs and add to the main list
    for chunk in file_chunks:
        old_id = chunk["id"]
        new_id = chunk_id_counter
        old_to_new_id_map[old_id] = new_id
        chunk["id"] = new_id
        chunks.append(chunk)
        chunk_id_counter += 1

    # Update chunk_id in questions and add to the main list
    for question in file_questions:
        old_chunk_id = question["chunk_id"]
        if old_chunk_id in old_to_new_id_map:
            question["chunk_id"] = old_to_new_id_map[old_chunk_id]
            questions.append(question)

print(f"Number of chunks    : {len(chunks)}")
print(f"Number of questions : {len(questions)}")

Number of chunks    : 120
Number of questions : 1209


## Create Qrels

[Qrels (Query Relevance Judgments)](https://amenra.github.io/ranx/qrels/) are the ground truth file or answer key used to evaluate a search system. They explicitly map each query to the list of documents that are considered the correct and relevant answers for that query.

In [32]:
from ranx import Qrels


def create_qrels_from_data(questions: list[dict]) -> Qrels:
    qrels_dict = {}
    for i, question in enumerate(questions):
        query_id = f"q_{i}"
        correct_chunk_id = question["chunk_id"]
        qrels_dict[query_id] = {str(correct_chunk_id): 1}

    return Qrels(qrels_dict)


qrels = create_qrels_from_data(questions)

## Create runs

A [Run](https://amenra.github.io/ranx/run/) is the output from your model. It shows which documents your model thinks are most relevant for each query, along with their similarity scores. These results are then compared to the Qrels (the answer key) to measure how well your model did.

In [33]:
import numpy as np

from ranx import Run
from sklearn.metrics.pairwise import cosine_similarity


def create_run_from_data(
    chunks: list[dict], questions: list[dict], model_name: str
) -> Run:
    run_dict = {}

    chunk_embeddings = np.array([chunk["embeddings"][model_name] for chunk in chunks])
    question_embeddings = np.array(
        [question["embeddings"][model_name] for question in questions]
    )
    similarity_scores = cosine_similarity(question_embeddings, chunk_embeddings)

    chunk_ids = [str(chunk["id"]) for chunk in chunks]
    for i, scores_for_one_question in enumerate(similarity_scores):
        chunk_scores = {
            chunk_id: score
            for chunk_id, score in zip(chunk_ids, scores_for_one_question)
        }

        query_id = f"q_{i}"
        run_dict[query_id] = chunk_scores

    return Run(run_dict, name=model_name)


runs = []
models_to_benchmark = list(chunks[0]["embeddings"].keys())

for current_model_name in models_to_benchmark:
    run = create_run_from_data(
        chunks=chunks, questions=questions, model_name=current_model_name
    )
    runs.append(run)

## Run the benchmark

Ranx handles the evaluation and statistical testing internally. You just need to call the [compare](https://amenra.github.io/ranx/compare/) function with the appropriate parameters.

In [34]:
from ranx import compare


metrics = ["mrr", "recall@1", "recall@5", "ndcg@5"]
report = compare(
    qrels=qrels,
    runs=runs,
    metrics=metrics,
    max_p=0.05,
    stat_test="fisher",
)

## Display results

### Metrics table

Assign a letter to each model for easy reference in the superscripts.

In [35]:
import string

model_to_letter = {}
for letter, name in zip(string.ascii_lowercase, report.model_names):
    model_to_letter[name] = letter

model_to_letter

{'text-embedding-3-small': 'a',
 'text-embedding-3-large': 'b',
 'all-minilm-l6-v2': 'c',
 'gemini-embedding-001': 'd',
 'qwen3-embedding-0.6b': 'e',
 'qwen3-embedding-8b': 'f',
 'qwen3-embedding-4b': 'g'}

Compute the cell data for each model and metric, including formatting the score and adding superscripts for statistical significance.

When you see something like this in a cell `0.8390 abcfg`, it means:
- `0.8390` is the score for that model on that metric.
- `abcfg` are the letters corresponding to other models that this model's score is statistically significantly better than.

The `max_widths` dictionary keeps track of the maximum width (in characters) for each column to make sure the output table lines up neatly and looks well-formatted. This helps each metric column fit its longest value, so the table is easy to read.

In [36]:
report.rounding_digits = 4
cell_data = {}
max_widths = {metric: len(metric) for metric in report.metrics}

for current_model_name in report.model_names:
    cell_data[current_model_name] = {}
    for metric in report.metrics:
        superscripts = ""
        current_model_score = report.results[current_model_name][metric]

        for other_model_name in report.model_names:
            if current_model_name == other_model_name:
                continue

            comparison_key = {current_model_name, other_model_name}
            p_value = report.comparisons[comparison_key][metric]["p_value"]
            other_model_score = report.results[other_model_name][metric]

            if current_model_score > other_model_score and p_value < report.max_p:
                superscripts += model_to_letter[other_model_name]

        score_str = f"{current_model_score:.{report.rounding_digits}f}"
        sorted_superscripts = "".join(sorted(superscripts))
        cell_data[current_model_name][metric] = (score_str, sorted_superscripts)

        total_len = len(score_str)
        if sorted_superscripts:
            space_len = 1
            total_len += space_len + len(sorted_superscripts)

        if total_len > max_widths[metric]:
            max_widths[metric] = total_len

In [37]:
print(max_widths)

{'mrr': 11, 'recall@1': 12, 'recall@5': 10, 'ndcg@5': 10}


In [38]:
cell_data

{'text-embedding-3-small': {'mrr': ('0.6972', 'c'),
  'recall@1': ('0.5790', 'c'),
  'recall@5': ('0.8544', 'c'),
  'ndcg@5': ('0.7261', 'c')},
 'text-embedding-3-large': {'mrr': ('0.6994', 'c'),
  'recall@1': ('0.5773', 'c'),
  'recall@5': ('0.8536', 'c'),
  'ndcg@5': ('0.7271', 'c')},
 'all-minilm-l6-v2': {'mrr': ('0.5396', ''),
  'recall@1': ('0.4268', ''),
  'recall@5': ('0.6576', ''),
  'ndcg@5': ('0.5501', '')},
 'gemini-embedding-001': {'mrr': ('0.7375', 'abc'),
  'recall@1': ('0.6303', 'abc'),
  'recall@5': ('0.8792', 'abc'),
  'ndcg@5': ('0.7639', 'abc')},
 'qwen3-embedding-0.6b': {'mrr': ('0.7276', 'abc'),
  'recall@1': ('0.6112', 'abc'),
  'recall@5': ('0.8801', 'abc'),
  'ndcg@5': ('0.7560', 'abc')},
 'qwen3-embedding-8b': {'mrr': ('0.7328', 'abc'),
  'recall@1': ('0.6129', 'abc'),
  'recall@5': ('0.8941', 'abc'),
  'ndcg@5': ('0.7643', 'abc')},
 'qwen3-embedding-4b': {'mrr': ('0.7425', 'abce'),
  'recall@1': ('0.6361', 'abcef'),
  'recall@5': ('0.8784', 'abc'),
  'ndcg@5':

Find the best scores for each metric to highlight them in the results table.

In [39]:
max_scores = {metric: -float("inf") for metric in report.metrics}
min_scores = {metric: float("inf") for metric in report.metrics}
for model_name in report.model_names:
    for metric in report.metrics:
        score = report.results[model_name][metric]
        if score > max_scores[metric]:
            max_scores[metric] = score
        if score < min_scores[metric]:
            min_scores[metric] = score

print(max_scores)
print(min_scores)

{'mrr': 0.742493532589355, 'recall@1': 0.6360628618693135, 'recall@5': 0.8941273779983457, 'ndcg@5': 0.767530255256212}
{'mrr': 0.5396133238351807, 'recall@1': 0.4267990074441687, 'recall@5': 0.6575682382133995, 'ndcg@5': 0.5500896918946327}


In [40]:
from typing import Any
from rich.box import HEAVY
from rich.text import Text
from rich.table import Table
from rich.console import Console


console = Console()
table = Table(show_header=True, header_style="bold bright_magenta", box=HEAVY)
table.add_column("#", style="dim")
table.add_column("Model", style="yellow")

for metric in report.metrics:
    table.add_column(metric, justify="left", width=max_widths[metric])

for model_name in report.model_names:
    row_data: list[Any] = [f"[dim]{model_to_letter[model_name]}[/dim]", model_name]

    for metric in report.metrics:
        cell_text = Text()
        score = report.results[model_name][metric]
        score_part, superscript_part = cell_data[model_name][metric]

        if score == max_scores[metric]:
            cell_text.append(score_part, style="bold bright_green")
        elif score == min_scores[metric]:
            cell_text.append(score_part, style="bold bright_red")
        else:
            cell_text.append(score_part)

        if superscript_part:
            cell_text.append(" ")
            cell_text.append(superscript_part, style="bold bright_cyan")

        row_data.append(cell_text)

    table.add_row(*row_data)

console.print(table)

┏━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ # ┃ Model                  ┃ mrr         ┃ recall@1     ┃ recall@5   ┃ ndcg@5     ┃
┣━━━╋━━━━━━━━━━━━━━━━━━━━━━━━╋━━━━━━━━━━━━━╋━━━━━━━━━━━━━━╋━━━━━━━━━━━━╋━━━━━━━━━━━━┫
┃ a ┃ text-embedding-3-small ┃ 0.6972 c    ┃ 0.5790 c     ┃ 0.8544 c   ┃ 0.7261 c   ┃
┃ b ┃ text-embedding-3-large ┃ 0.6994 c    ┃ 0.5773 c     ┃ 0.8536 c   ┃ 0.7271 c   ┃
┃ c ┃ all-minilm-l6-v2       ┃ 0.5396      ┃ 0.4268       ┃ 0.6576     ┃ 0.5501     ┃
┃ d ┃ gemini-embedding-001   ┃ 0.7375 abc  ┃ 0.6303 abc   ┃ 0.8792 abc ┃ 0.7639 abc ┃
┃ e ┃ qwen3-embedding-0.6b   ┃ 0.7276 abc  ┃ 0.6112 abc   ┃ 0.8801 abc ┃ 0.7560 abc ┃
┃ f ┃ qwen3-embedding-8b     ┃ 0.7328 abc  ┃ 0.6129 abc   ┃ 0.8941 abc ┃ 0.7643 abc ┃
┃ g ┃ qwen3-embedding-4b     ┃ 0.7425 abce ┃ 0.6361 abcef ┃ 0.8784 abc ┃ 0.7675 abc ┃
┗━━━┻━━━━━━━━━━━━━━━━━━━━━━━━┻━━━━━━━━━━━━━┻━━━━━━━━━━━━━━┻━━━━━━━━━━━━┻━━━━━━━━━━━━┛

If you want to sort the models by their average score across all metrics, you can calculate the average for each model and then sort them accordingly. This way, you can see which models perform best overall, rather than just on individual metrics.

In [41]:
from typing import Any
from rich.box import HEAVY
from rich.text import Text
from rich.table import Table
from rich.console import Console


console = Console()
table = Table(show_header=True, header_style="bold bright_magenta", box=HEAVY)
table.add_column("#", style="dim")
table.add_column("Model", style="yellow")

for metric in report.metrics:
    table.add_column(metric, justify="left", width=max_widths[metric])

table.add_column("Average", justify="right", style="bold cyan")

model_averages = {}
for model_name in report.model_names:
    avg_score = np.mean(
        [report.results[model_name][metric] for metric in report.metrics]
    )
    model_averages[model_name] = avg_score

sorted_models = sorted(model_averages.items(), key=lambda x: x[1], reverse=True)
best_model_name = sorted_models[0][0]

for model_name, avg_score in sorted_models:
    model_name_cell_value = model_name
    if model_name == best_model_name:
        model_name_cell_value = f"[bold bright_green]{model_name}[/bold bright_green]"

    row_data: list[Any] = [
        f"[dim]{model_to_letter[model_name]}[/dim]",
        model_name_cell_value,
    ]

    for metric in report.metrics:
        cell_text = Text()
        score = report.results[model_name][metric]
        score_part, superscript_part = cell_data[model_name][metric]

        if score == max_scores[metric]:
            cell_text.append(score_part, style="bold bright_green")
        elif score == min_scores[metric]:
            cell_text.append(score_part, style="bold bright_red")
        else:
            cell_text.append(score_part)

        if superscript_part:
            cell_text.append(" ")
            cell_text.append(superscript_part, style="bold bright_cyan")

        row_data.append(cell_text)

    row_data.append(f"{avg_score:.4f}")
    table.add_row(*row_data)

console.print(table)

┏━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━┓
┃ # ┃ Model                  ┃ mrr         ┃ recall@1     ┃ recall@5   ┃ ndcg@5     ┃ Average ┃
┣━━━╋━━━━━━━━━━━━━━━━━━━━━━━━╋━━━━━━━━━━━━━╋━━━━━━━━━━━━━━╋━━━━━━━━━━━━╋━━━━━━━━━━━━╋━━━━━━━━━┫
┃ g ┃ qwen3-embedding-4b     ┃ 0.7425 abce ┃ 0.6361 abcef ┃ 0.8784 abc ┃ 0.7675 abc ┃  0.7561 ┃
┃ d ┃ gemini-embedding-001   ┃ 0.7375 abc  ┃ 0.6303 abc   ┃ 0.8792 abc ┃ 0.7639 abc ┃  0.7527 ┃
┃ f ┃ qwen3-embedding-8b     ┃ 0.7328 abc  ┃ 0.6129 abc   ┃ 0.8941 abc ┃ 0.7643 abc ┃  0.7510 ┃
┃ e ┃ qwen3-embedding-0.6b   ┃ 0.7276 abc  ┃ 0.6112 abc   ┃ 0.8801 abc ┃ 0.7560 abc ┃  0.7437 ┃
┃ b ┃ text-embedding-3-large ┃ 0.6994 c    ┃ 0.5773 c     ┃ 0.8536 c   ┃ 0.7271 c   ┃  0.7144 ┃
┃ a ┃ text-embedding-3-small ┃ 0.6972 c    ┃ 0.5790 c     ┃ 0.8544 c   ┃ 0.7261 c   ┃  0.7142 ┃
┃ c ┃ all-minilm-l6-v2       ┃ 0.5396      ┃ 0.4268       ┃ 0.6576     ┃ 0.5501     ┃  0.5435 ┃
┗━━━┻━━━━━━━━━━━━━━━━━━━━━━━━┻━━━━━━━━━━━━━┻━━━━━━━━━━━━━━┻━━━━━━━━━━━━┻━━━━━━━━━━━━┻━━━━━━━━━┛

### Win Tie Loss table

The `win_tie_loss` dictionary shows how many times a model won, tied, or lost against other models based on the statistical significance of their scores.

Here is an example of an object from the `win_tie_loss` dictionary:

```json
"('all-minilm-l6-v2', 'qwen3-embedding-0.6b')": {
    "mrr": {
        "W": 37,
        "T": 229,
        "L": 111
    },
    "recall@1": {
        "W": 15,
        "T": 307,
        "L": 55
    },
    "recall@5": {
        "W": 10,
        "T": 331,
        "L": 36
    },
    "ndcg@5": {
        "W": 29,
        "T": 252,
        "L": 96
    }
}
```

The keys are tuples representing pairs of models being compared. The values are dictionaries where each key is a metric (like `mrr`, `recall@1`, etc.) and the value is another dictionary with counts of Wins (`W`), Ties (`T`), and Losses (`L`) for the first model against the second model.

In [42]:
import json


def tuple_keys_to_str(dictionary: dict) -> dict:
    return {str(k): v for k, v in dictionary.items()}


print(json.dumps(tuple_keys_to_str(report.win_tie_loss), indent=2))

{
  "('text-embedding-3-small', 'text-embedding-3-large')": {
    "mrr": {
      "W": 235,
      "T": 725,
      "L": 249
    },
    "recall@1": {
      "W": 82,
      "T": 1047,
      "L": 80
    },
    "recall@5": {
      "W": 54,
      "T": 1102,
      "L": 53
    },
    "ndcg@5": {
      "W": 184,
      "T": 840,
      "L": 185
    }
  },
  "('text-embedding-3-small', 'all-minilm-l6-v2')": {
    "mrr": {
      "W": 570,
      "T": 457,
      "L": 182
    },
    "recall@1": {
      "W": 279,
      "T": 835,
      "L": 95
    },
    "recall@5": {
      "W": 283,
      "T": 881,
      "L": 45
    },
    "ndcg@5": {
      "W": 477,
      "T": 584,
      "L": 148
    }
  },
  "('text-embedding-3-small', 'gemini-embedding-001')": {
    "mrr": {
      "W": 216,
      "T": 676,
      "L": 317
    },
    "recall@1": {
      "W": 92,
      "T": 963,
      "L": 154
    },
    "recall@5": {
      "W": 70,
      "T": 1039,
      "L": 100
    },
    "ndcg@5": {
      "W": 186,
      "T": 746,
  

Create a table that displays the Win, Tie, and Loss counts for each pair of models across all metrics.

In [43]:
console = Console()
table = Table(show_header=True, header_style="bold bright_magenta", box=HEAVY)
table.add_column("Model A", style="yellow")
table.add_column("Model B", style="yellow")
table.add_column("Metric", style="white")
table.add_column("Wins (A)", justify="center", style="bold bright_green")
table.add_column("Ties", justify="center", style="grey70")
table.add_column("Losses (A)", justify="center", style="bold bright_red")
table.add_column("p-value", justify="right", style="bold")

win_tie_loss = report.win_tie_loss
shown_pairs = set()

for key, metrics_data in win_tie_loss.items():
    model_a, model_b = key  # type: ignore
    pair = frozenset([model_a, model_b])
    if pair in shown_pairs:
        continue
    shown_pairs.add(pair)

    for metric, scores in metrics_data.items():
        p_value = report.comparisons[key][metric]["p_value"]
        p_value_style = "bold bright_green" if p_value < report.max_p else "white"

        if scores["W"] > scores["L"]:
            model_a_text = Text(model_a, style="bold bright_green")
            model_b_text = Text(model_b)
        elif scores["L"] > scores["W"]:
            model_a_text = Text(model_a)
            model_b_text = Text(model_b, style="bold bright_green")
        else:
            model_a_text = Text(model_a)
            model_b_text = Text(model_b)

        table.add_row(
            model_a_text,
            model_b_text,
            metric,
            str(scores["W"]),
            str(scores["T"]),
            str(scores["L"]),
            Text(f"{p_value:.3f}", style=p_value_style),
        )

    table.add_row("")

console.print(table)

┏━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━┓
┃ Model A                ┃ Model B                ┃ Metric   ┃ Wins (A) ┃ Ties ┃ Losses (A) ┃ p-value ┃
┣━━━━━━━━━━━━━━━━━━━━━━━━╋━━━━━━━━━━━━━━━━━━━━━━━━╋━━━━━━━━━━╋━━━━━━━━━━╋━━━━━━╋━━━━━━━━━━━━╋━━━━━━━━━┫
┃ text-embedding-3-small ┃ text-embedding-3-large ┃ mrr      ┃   235    ┃ 725  ┃    249     ┃   0.756 ┃
┃ text-embedding-3-small ┃ text-embedding-3-large ┃ recall@1 ┃    82    ┃ 1047 ┃     80     ┃   0.946 ┃
┃ text-embedding-3-small ┃ text-embedding-3-large ┃ recall@5 ┃    54    ┃ 1102 ┃     53     ┃   1.000 ┃
┃ text-embedding-3-small ┃ text-embedding-3-large ┃ ndcg@5   ┃   184    ┃ 840  ┃    185     ┃   0.889 ┃
┃                        ┃                        ┃          ┃          ┃      ┃            ┃         ┃
┃ text-embedding-3-small ┃ all-minilm-l6-v2       ┃ mrr      ┃   570    ┃ 457  ┃    182     ┃   0.000 ┃
┃ text-embedding-3-small ┃ all-minilm-l6-v2       ┃ recall@1 ┃   279    ┃ 835  ┃     95     ┃   0.000 ┃
┃ text-embedding-3-small ┃ all-minilm-l6-v2       ┃ recall@5 ┃   283    ┃ 881  ┃     45     ┃   0.000 ┃
┃ text-embedding-3-small ┃ all-minilm-l6-v2       ┃ ndcg@5   ┃   477    ┃ 584  ┃    148     ┃   0.000 ┃
┃                        ┃                        ┃          ┃          ┃      ┃            ┃         ┃
┃ text-embedding-3-small ┃ gemini-embedding-001   ┃ mrr      ┃   216    ┃ 676  ┃    317     ┃   0.000 ┃
┃ text-embedding-3-small ┃ gemini-embedding-001   ┃ recall@1 ┃    92    ┃ 963  ┃    154     ┃   0.000 ┃
┃ text-embedding-3-small ┃ gemini-embedding-001   ┃ recall@5 ┃    70    ┃ 1039 ┃    100     ┃   0.027 ┃
┃ text-embedding-3-small ┃ gemini-embedding-001   ┃ ndcg@5   ┃   186    ┃ 746  ┃    277     ┃   0.000 ┃
┃                        ┃                        ┃          ┃          ┃      ┃            ┃         ┃
┃ text-embedding-3-small ┃ qwen3-embedding-0.6b   ┃ mrr      ┃   243    ┃ 654  ┃    312     ┃   0.000 ┃
┃ text-embedding-3-small ┃ qwen3-embedding-0.6b   ┃ recall@1 ┃   113    ┃ 944  ┃    152     ┃   0.019 ┃
┃ text-embedding-3-small ┃ qwen3-embedding-0.6b   ┃ recall@5 ┃    72    ┃ 1034 ┃    103     ┃   0.021 ┃
┃ text-embedding-3-small ┃ qwen3-embedding-0.6b   ┃ ndcg@5   ┃   215    ┃ 722  ┃    272     ┃   0.000 ┃
┃                        ┃                        ┃          ┃          ┃      ┃            ┃         ┃
┃ text-embedding-3-small ┃ qwen3-embedding-8b     ┃ mrr      ┃   225    ┃ 658  ┃    326     ┃   0.000 ┃
┃ text-embedding-3-small ┃ qwen3-embedding-8b     ┃ recall@1 ┃   106    ┃ 956  ┃    147     ┃   0.012 ┃
┃ text-embedding-3-small ┃ qwen3-embedding-8b     ┃ recall@5 ┃    67    ┃ 1027 ┃    115     ┃   0.001 ┃
┃ text-embedding-3-small ┃ qwen3-embedding-8b     ┃ ndcg@5   ┃   205    ┃ 716  ┃    288     ┃   0.000 ┃
┃                        ┃                        ┃          ┃          ┃      ┃            ┃         ┃
┃ text-embedding-3-small ┃ qwen3-embedding-4b     ┃ mrr      ┃   217    ┃ 677  ┃    315     ┃   0.000 ┃
┃ text-embedding-3-small ┃ qwen3-embedding-4b     ┃ recall@1 ┃    88    ┃ 964  ┃    157     ┃   0.000 ┃
┃ text-embedding-3-small ┃ qwen3-embedding-4b     ┃ recall@5 ┃    87    ┃ 1006 ┃    116     ┃   0.035 ┃
┃ text-embedding-3-small ┃ qwen3-embedding-4b     ┃ ndcg@5   ┃   192    ┃ 732  ┃    285     ┃   0.000 ┃
┃                        ┃                        ┃          ┃          ┃      ┃            ┃         ┃
┃ text-embedding-3-large ┃ all-minilm-l6-v2       ┃ mrr      ┃   566    ┃ 472  ┃    171     ┃   0.000 ┃
┃ text-embedding-3-large ┃ all-minilm-l6-v2       ┃ recall@1 ┃   268    ┃ 855  ┃     86     ┃   0.000 ┃
┃ text-embedding-3-large ┃ all-minilm-l6-v2       ┃ recall@5 ┃   281    ┃ 884  ┃     44     ┃   0.000 ┃
┃ text-embedding-3-large ┃ all-minilm-l6-v2       ┃ ndcg@5   ┃   468    ┃ 600  ┃    141     ┃   0.000 ┃
┃                        ┃                        ┃          ┃          ┃      ┃            ┃         ┃
┃ text-embedding-3-large ┃ gemini-embedding-001 